In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 3_anisotropic_df_likelihoods.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Compute likelihoods for the anisotropic DFs.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, dill as pickle
import pdb, copy, glob, time

## Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu
from astropy import constants as apc

## Analysis
import scipy.stats
import scipy.interpolate

## galpy
from galpy import orbit
from galpy import potential
from galpy import actionAngle as aA
from galpy import df
from galpy import util as gputil

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import kinematics as pkin
from tng_dfs import util as putil
from tng_dfs import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','RO','VO','ZO','LITTLE_H',
            'MW_MASS_RANGE']
data_dir,mw_analog_dir,ro,vo,zo,h,mw_mass_range = \
    putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
# epsen_fig_dir = '/epsen_data/scr/lane/projects/tng-dfs/figs/notebooks/sample/'
# os.makedirs(epsen_fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Compute DF values for the constant beta DFs

In [ ]:
verbose = True
epsen_dens_fitting_dir = '/epsen_data/scr/lane/projects/tng-dfs/fitting/'+\
    'density_profile/'
epsen_df_fitting_dir = '/epsen_data/scr/lane/projects/tng-dfs/fitting/'+\
    'distribution_function/'
fig_dir = './fig/constant_beta/'
os.makedirs(fig_dir,exist_ok=True)

# DM halo information
dm_halo_version = 'poisson_nfw'
dm_halo_ncut = 500

# Stellar bulge and disk information
stellar_bulge_disk_version = 'miyamoto_disk_pswc_bulge_tps_halo'
stellar_bulge_disk_ncut = 2000

# Stellar halo density information
stellar_halo_density_version = 'poisson_twopower'
stellar_halo_density_ncut = 500

# Stellar halo rotation information
stellar_halo_rotation_version = 'tanh_rotation'
stellar_halo_rotation_ncut = 500

# Beta information
beta_version = 'constant_beta'
beta_ncut = 500

# Define density profiles
dm_halo_densfunc = pdens.NFWSpherical()
disk_densfunc = pdens.MiyamotoNagaiDisk()
bulge_densfunc = pdens.SinglePowerCutoffSpherical()
stellar_halo_densfunc = pdens.TwoPowerSpherical()
stellar_bulge_disk_densfunc = pdens.CompositeDensityProfile(
    [disk_densfunc,
     bulge_densfunc,
     stellar_halo_densfunc]
     )

df_vals_cb = []
df_vals_self_cb = []

for i in range(n_mw):
    # if i != 2: continue
    if verbose: print(f'Plotting MW {i+1}/{n_mw}')

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    # # Get the dark halo
    dm_halo_filename = os.path.join(epsen_dens_fitting_dir,'dm_halo/',dm_halo_version,
        str(z0_sid), 'sampler.pkl')
    dm_halo_pot = pfit.construct_pot_from_fit(dm_halo_filename,
        dm_halo_densfunc, dm_halo_ncut, ro=ro, vo=vo)
    
    # Get the stellar bulge and disk
    stellar_bulge_disk_filename = os.path.join(epsen_dens_fitting_dir,
        'stellar_bulge_disk/',stellar_bulge_disk_version,str(z0_sid),
        'sampler.pkl')
    stellar_pots = pfit.construct_pot_from_fit(stellar_bulge_disk_filename,
        stellar_bulge_disk_densfunc, stellar_bulge_disk_ncut, ro=ro, vo=vo)
    fpot = [stellar_pots[1], stellar_pots[0], dm_halo_pot] # bulge, disk, halo

    # Load the interpolator for the sphericalized potential
    interpolator_filename = os.path.join(epsen_dens_fitting_dir,
        'spherical_interpolated_potential/',str(z0_sid),'interp_potential.pkl')
    with open(interpolator_filename,'rb') as handle:
        interpot = pickle.load(handle)

    _df_vals = []
    _df_vals_self = []

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'  Constructing DF for major merger {j+1}/{n_major}')

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]

        # Get energy and angular momentum
        orbs = co.get_orbs('stars')[indx]
        n_star = len(orbs)
        rs = orbs.r().to_value(apu.kpc)
        masses = co.get_masses('stars')[indx].to_value(apu.Msun)
        L = np.sum(orbs.L().to_value(apu.kpc*apu.km/apu.s)**2,axis=1)**0.5
        Lz = orbs.Lz().to_value(apu.kpc*apu.km/apu.s)
        pe = co.get_potential_energy('stars')[indx].to_value(apu.km**2/apu.s**2)
        vels = co.get_velocities('stars')[indx].to_value(apu.km/apu.s)
        vmag = np.linalg.norm(vels,axis=1)
        energy = pe + 0.5*vmag**2

        # Get the stellar halo density profile (denspot for the DF)
        stellar_halo_density_filename = os.path.join(epsen_dens_fitting_dir,
            'stellar_halo/',stellar_halo_density_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        denspot = pfit.construct_pot_from_fit(
            stellar_halo_density_filename, stellar_halo_densfunc, 
            stellar_halo_density_ncut, ro=ro, vo=vo)
        
        # Get the stellar halo rotation kernel
        stellar_halo_rotation_filename = os.path.join(epsen_dens_fitting_dir,
            'stellar_halo/',stellar_halo_rotation_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        assert os.path.exists(stellar_halo_rotation_filename)
        with open(stellar_halo_rotation_filename,'rb') as handle:
            stellar_halo_rotation_sampler = pickle.load(handle)
        stellar_halo_rotation_samples = stellar_halo_rotation_sampler.get_chain(
            discard=stellar_halo_rotation_ncut, flat=True)
        # Params are frot, chi
        stellar_halo_frot, stellar_halo_chi = \
            np.median(stellar_halo_rotation_samples,axis=0)
        
        # Load the distribution function and wrangle
        df_filename = os.path.join(epsen_df_fitting_dir,beta_version,
            str(z0_sid),'merger_'+str(j+1),'df.pkl')
        with open(df_filename,'rb') as handle:
            dfcb = pickle.load(handle)
        dfcb = pkin.reconstruct_anisotropic_df(dfcb, interpot, denspot)

        # Scale the input energies by the potential energy of interpot at the 
        # stellar half-mass radius
        rhalf = pkin.half_mass_radius(rs, masses)
        rhalf_perc = 0.05
        rhalf_mask = np.abs(rs-rhalf) < rhalf_perc*rhalf
        rhalf_pe_star = np.median(pe[rhalf_mask])
        rhalf_pe_interpot = potential.evaluatePotentials(
            interpot, rhalf*apu.kpc, 0.).to_value(apu.km**2/apu.s**2)
        rhalf_pe_offset = rhalf_pe_star - rhalf_pe_interpot

        # Compute the internal energy and angular momentum
        energy_offset = energy - rhalf_pe_offset
        energy_offset_internal = energy_offset / (vo**2)
        L_internal = L/ro/vo
        
        # Compute the value of the DF speedily using the interpolator
        fE = dfcb._fE_interp(energy_offset_internal)
        fL = L_internal**(-2*dfcb._beta)
        fLz = (stellar_halo_frot*np.tanh(Lz/stellar_halo_chi)-stellar_halo_frot+1)
        f = fE*fL*fLz

        # Do the self-similar assessment of the DF
        # Create the samples for the DF
        sample = dfcb.sample(n=n_star, rmin=rs.min()*apu.kpc*0.9)
        sample = pkin.rotate_df_samples(sample,stellar_halo_frot,stellar_halo_chi)

        # Compute the likelihoods for the self-similar DF
        pe_offset_sample = 0. # PE offset for DF samples should be 0
        energy_offset_sample = sample.E(pot=interpot).to_value(apu.km**2/apu.s**2) - pe_offset_sample
        energy_offset_internal_sample = energy_offset_sample / (vo**2)
        L_sample = (np.sum(sample.L()**2,1)**0.5).to_value(apu.kpc*apu.km/apu.s)
        L_internal_sample = L_sample/ro/vo
        Lz_sample = sample.Lz().to_value(apu.kpc*apu.km/apu.s)
        fE_sample = dfcb._fE_interp(energy_offset_internal_sample)
        fL_sample = L_internal_sample**(-2*dfcb._beta)
        fLz_sample = (stellar_halo_frot*np.tanh(Lz_sample/stellar_halo_chi)-stellar_halo_frot+1)
        f_sample = fE_sample*fL_sample*fLz_sample

        # Save the values
        _df_vals.append([f,fE,fL,fLz,z0_sid,j+1,major_mlpid])
        _df_vals_self.append([f_sample,fE_sample,fL_sample,fLz_sample,z0_sid,j+1,major_mlpid])

    df_vals_cb.append( _df_vals )
    df_vals_self_cb.append( _df_vals_self )

os.makedirs('./data/', exist_ok=True)
with open('./data/dfvals_cb.pkl','wb') as handle:
    pickle.dump(df_vals_cb, handle)
with open('./data/dfvals_self_cb.pkl','wb') as handle:
    pickle.dump(df_vals_self_cb, handle)

### Compute DF values for the Osipkov-Merritt DFs

In [ ]:
verbose = True
epsen_dens_fitting_dir = '/epsen_data/scr/lane/projects/tng-dfs/fitting/'+\
    'density_profile/'
epsen_df_fitting_dir = '/epsen_data/scr/lane/projects/tng-dfs/fitting/'+\
    'distribution_function/'
fig_dir = './fig/constant_beta/'
os.makedirs(fig_dir,exist_ok=True)

# DM halo information
dm_halo_version = 'poisson_nfw'
dm_halo_ncut = 500

# Stellar bulge and disk information
stellar_bulge_disk_version = 'miyamoto_disk_pswc_bulge_tps_halo'
stellar_bulge_disk_ncut = 2000

# Stellar halo density information
stellar_halo_density_version = 'poisson_twopower'
stellar_halo_density_ncut = 500

# Stellar halo rotation information
stellar_halo_rotation_version = 'tanh_rotation'
stellar_halo_rotation_ncut = 500

# Beta information
beta_version = 'osipkov_merritt'
beta_ncut = 500

# Define density profiles
dm_halo_densfunc = pdens.NFWSpherical()
disk_densfunc = pdens.MiyamotoNagaiDisk()
bulge_densfunc = pdens.SinglePowerCutoffSpherical()
stellar_halo_densfunc = pdens.TwoPowerSpherical()
stellar_bulge_disk_densfunc = pdens.CompositeDensityProfile(
    [disk_densfunc,
     bulge_densfunc,
     stellar_halo_densfunc]
     )

df_vals_om = []
df_vals_self_om = []

for i in range(n_mw):
    # if i != 0: continue
    if verbose: print(f'Plotting MW {i+1}/{n_mw}')

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    # # Get the dark halo
    dm_halo_filename = os.path.join(epsen_dens_fitting_dir,'dm_halo/',dm_halo_version,
        str(z0_sid), 'sampler.pkl')
    dm_halo_pot = pfit.construct_pot_from_fit(dm_halo_filename,
        dm_halo_densfunc, dm_halo_ncut, ro=ro, vo=vo)
    
    # Get the stellar bulge and disk
    stellar_bulge_disk_filename = os.path.join(epsen_dens_fitting_dir,
        'stellar_bulge_disk/',stellar_bulge_disk_version,str(z0_sid),
        'sampler.pkl')
    stellar_pots = pfit.construct_pot_from_fit(stellar_bulge_disk_filename,
        stellar_bulge_disk_densfunc, stellar_bulge_disk_ncut, ro=ro, vo=vo)
    fpot = [stellar_pots[1], stellar_pots[0], dm_halo_pot] # bulge, disk, halo

    # Load the interpolator for the sphericalized potential
    interpolator_filename = os.path.join(epsen_dens_fitting_dir,
        'spherical_interpolated_potential/',str(z0_sid),'interp_potential.pkl')
    with open(interpolator_filename,'rb') as handle:
        interpot = pickle.load(handle)

    _df_vals = []
    _df_vals_self = []

    for j in range(n_major):
        # if j > 0: continue
        if verbose: print(f'  Constructing DF for major merger {j+1}/{n_major}')

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]

        # Get energy and angular momentum
        orbs = co.get_orbs('stars')[indx]
        n_star = len(orbs)
        rs = orbs.r().to_value(apu.kpc)
        masses = co.get_masses('stars')[indx].to_value(apu.Msun)
        L = np.sum(orbs.L().to_value(apu.kpc*apu.km/apu.s)**2,axis=1)**0.5
        Lz = orbs.Lz().to_value(apu.kpc*apu.km/apu.s)
        pe = co.get_potential_energy('stars')[indx].to_value(apu.km**2/apu.s**2)
        vels = co.get_velocities('stars')[indx].to_value(apu.km/apu.s)
        vmag = np.linalg.norm(vels,axis=1)
        energy = pe + 0.5*vmag**2

        # Get the stellar halo density profile (denspot for the DF)
        stellar_halo_density_filename = os.path.join(epsen_dens_fitting_dir,
            'stellar_halo/',stellar_halo_density_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        denspot = pfit.construct_pot_from_fit(
            stellar_halo_density_filename, stellar_halo_densfunc, 
            stellar_halo_density_ncut, ro=ro, vo=vo)
        
        # Get the stellar halo rotation kernel
        stellar_halo_rotation_filename = os.path.join(epsen_dens_fitting_dir,
            'stellar_halo/',stellar_halo_rotation_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        assert os.path.exists(stellar_halo_rotation_filename)
        with open(stellar_halo_rotation_filename,'rb') as handle:
            stellar_halo_rotation_sampler = pickle.load(handle)
        stellar_halo_rotation_samples = stellar_halo_rotation_sampler.get_chain(
            discard=stellar_halo_rotation_ncut, flat=True)
        # Params are frot, chi
        stellar_halo_frot, stellar_halo_chi = \
            np.median(stellar_halo_rotation_samples,axis=0)
        
        # Load the distribution function and wrangle
        df_filename = os.path.join(epsen_df_fitting_dir,beta_version,
            str(z0_sid),'merger_'+str(j+1),'df.pkl')
        with open(df_filename,'rb') as handle:
            dfom = pickle.load(handle)
        dfom = pkin.reconstruct_anisotropic_df(dfom, interpot, denspot)

        # Scale the input energies by the potential energy of interpot at the 
        # stellar half-mass radius
        rhalf = pkin.half_mass_radius(rs, masses)
        rhalf_perc = 0.05
        rhalf_mask = np.abs(rs-rhalf) < rhalf_perc*rhalf
        rhalf_pe_star = np.median(pe[rhalf_mask])
        rhalf_pe_interpot = potential.evaluatePotentials(
            interpot, rhalf*apu.kpc, 0.).to_value(apu.km**2/apu.s**2)
        rhalf_pe_offset = rhalf_pe_star - rhalf_pe_interpot

        # Compute the energy and Q
        energy_offset = energy - rhalf_pe_offset
        Q_offset = -energy_offset - 0.5*L**2/(dfom._ra*ro)**2
        Q_offset_internal = Q_offset / (vo**2)
        
        # Compute the value of the DF speedily using the interpolator
        fQ = np.exp( dfom._logfQ_interp(Q_offset_internal) )
        fLz = (stellar_halo_frot*np.tanh(Lz/stellar_halo_chi)-stellar_halo_frot+1)
        f = fQ*fLz

        # Do the self-similar assessment of the DF
        # Create the samples for the DF
        sample = dfom.sample(n=n_star, rmin=rs.min()*apu.kpc*0.9)
        sample = pkin.rotate_df_samples(sample,stellar_halo_frot,stellar_halo_chi)

        # Compute the likelihoods for the self-similar DF
        pe_offset_sample = 0. # PE offset for DF samples should be 0
        energy_offset_sample = sample.E(pot=interpot).to_value(apu.km**2/apu.s**2) - pe_offset_sample
        L_sample = (np.sum(sample.L()**2,1)**0.5).to_value(apu.kpc*apu.km/apu.s)
        Q_offset_sample = -energy_offset_sample - 0.5*L_sample**2/(dfom._ra*ro)**2
        Q_offset_internal_sample = Q_offset_sample / (vo**2)
        Lz_sample = sample.Lz().to_value(apu.kpc*apu.km/apu.s)
        fQ_sample = np.exp( dfom._logfQ_interp(Q_offset_internal_sample) )
        fLz_sample = (stellar_halo_frot*np.tanh(Lz_sample/stellar_halo_chi)-stellar_halo_frot+1)
        f_sample = fQ_sample*fLz_sample

        _df_vals.append([f,fQ,fLz,z0_sid,j+1,major_mlpid])
        _df_vals_self.append([f_sample,fQ_sample,fLz_sample,z0_sid,j+1,major_mlpid])

    df_vals_om.append( _df_vals )
    df_vals_self_om.append( _df_vals_self )

os.makedirs('./data/', exist_ok=True)
with open('./data/dfvals_om.pkl','wb') as handle:
    pickle.dump(df_vals_om, handle)
with open('./data/dfvals_self_om.pkl','wb') as handle:
    pickle.dump(df_vals_self_om, handle)

### Wrangle the data

In [ ]:
# All DF vals should have been saved if already calculated, so don't need to 
# ask permission to overwrite
with open('./data/dfvals_cb.pkl','rb') as handle:
    df_vals_cb = pickle.load(handle)
with open('./data/dfvals_self_cb.pkl','rb') as handle:
    df_vals_self_cb = pickle.load(handle)

with open('./data/dfvals_om.pkl','rb') as handle:
    df_vals_om = pickle.load(handle)
with open('./data/dfvals_self_om.pkl','rb') as handle:
    df_vals_self_om = pickle.load(handle)

In [ ]:
# Get the input star and dark matter masses
if not os.path.exists('./data/star_mass.pkl') or not os.path.exists('./data/dm_mass.pkl'):
    star_mass = []
    dm_mass = []

    verbose = True

    for i in range(n_mw):
        # if i > 1: continue
        if verbose: print(f'Getting MW {i+1}/{n_mw}')

        # Get the primary
        primary = tree_primaries[i]
        z0_sid = primary.subfind_id[0]
        n_snap = len(primary.snapnum)
        n_major = primary.n_major_mergers
        primary_filename = primary.get_cutout_filename(mw_analog_dir,
            snapnum=primary.snapnum[0])
        co = pcutout.TNGCutout(primary_filename)
        dmpid = co.get_property('dm','ParticleIDs')
        dmass = co.get_masses('dm').to_value(apu.Msun)
        spid = co.get_property('stars','ParticleIDs')
        smass = co.get_masses('stars').to_value(apu.Msun)

        _star_mass = []
        _dm_mass = []

        for j in range(n_major):
            if verbose: print(f'Merger {j+1}/{n_major}')

            # Get the major merger particle IDs and mask
            major_merger = primary.tree_major_mergers[j]
            dmupid = major_merger.get_unique_particle_ids('dm',data_dir=data_dir)
            supid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
            dmindx = np.isin(dmpid, dmupid)
            sindx = np.isin(spid, supid)

            _star_mass.append( np.sum(smass[sindx]) )
            _dm_mass.append( np.sum(dmass[dmindx]) )
        
        star_mass.append(_star_mass)
        dm_mass.append(_dm_mass)

else:
    with open('./data/star_mass.pkl','rb') as handle:
        star_mass = pickle.load(handle)
    with open('./data/dm_mass.pkl','rb') as handle:
        dm_mass = pickle.load(handle)

In [ ]:
### Wrangle the input loglikelihoods a bit

loglike_data = []
indx = 0

for i in range(n_mw):
    # if i > 5: continue
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    n_snap = len(primary.snapnum)
    n_major = primary.n_major_mergers

    _loglike_data = []

    for j in range(n_major):
        
        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_mlpid = major_merger.secondary_mlpid

        f_cb = df_vals_cb[i][j][0]
        f_om = df_vals_om[i][j][0]
        f_cb_self = df_vals_self_cb[i][j][0]
        f_om_self = df_vals_self_om[i][j][0]

        f_cb_mask = f_cb > 0
        f_om_mask = f_om > 0
        f_cb_self_mask = f_cb_self > 0
        f_om_self_mask = f_om_self > 0
        f_mask = f_cb_mask & f_om_mask & f_cb_self_mask & f_om_self_mask

        _loglike_cb = np.log(f_cb[f_mask])
        _loglike_om = np.log(f_om[f_mask])
        _loglike_cb_self = np.log(f_cb_self[f_mask])
        _loglike_om_self = np.log(f_om_self[f_mask])

        assert z0_sid == df_vals_cb[i][j][4]
        assert major_mlpid == df_vals_cb[i][j][6]

        _loglike_data.append( [_loglike_cb, _loglike_om, _loglike_cb_self, 
            _loglike_om_self, z0_sid, j+1, major_mlpid] )
    
    loglike_data.append( _loglike_data )

In [ ]:
# Compute the loglike differences
loglike_diff_data = []

for i in range(n_mw):
    # if i > 5: continue
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    n_snap = len(primary.snapnum)
    n_major = primary.n_major_mergers

    _loglike_diff_data = []

    for j in range(n_major):

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_mlpid = major_merger.secondary_mlpid

        assert loglike_data[i][j][4] == z0_sid
        assert loglike_data[i][j][5] == j+1
        assert loglike_data[i][j][6] == major_mlpid

        lld_cb_om = np.sum(loglike_data[i][j][0] - loglike_data[i][j][1])
        lld_cb_self = np.sum(loglike_data[i][j][0] - loglike_data[i][j][2])
        lld_om_self = np.sum(loglike_data[i][j][1] - loglike_data[i][j][3])
        lld_cb_om_self = np.sum((loglike_data[i][j][0] - loglike_data[i][j][2]) - \
                                (loglike_data[i][j][1] - loglike_data[i][j][3]))
        lld_N = len(loglike_data[i][j][0])
        
        _loglike_diff_data.append([lld_cb_om, 
                                   lld_cb_self, 
                                   lld_om_self,
                                   lld_cb_om_self, 
                                   lld_N
                                   ])

    loglike_diff_data.append(_loglike_diff_data)

In [ ]:
### Finally construct a structured numpy array

# Compute the loglike differences
data = []

for i in range(n_mw):
    # if i > 5: continue
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    n_snap = len(primary.snapnum)
    n_major = primary.n_major_mergers

    for j in range(n_major):

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_mlpid = major_merger.secondary_mlpid

        assert loglike_data[i][j][4] == z0_sid
        assert loglike_data[i][j][5] == j+1
        assert loglike_data[i][j][6] == major_mlpid

        _data = (loglike_diff_data[i][j][0], # CB - OM
                 loglike_diff_data[i][j][1], # CB - CB_self
                 loglike_diff_data[i][j][2], # OM - OM_self
                 loglike_diff_data[i][j][3], # (CB - CB_self) - (OM - OM_self)
                 loglike_diff_data[i][j][4], # loglike N
                 star_mass[i][j], 
                 dm_mass[i][j],
                 major_merger.star_mass_ratio,
                 major_merger.dm_mass_ratio,
                 putil.snapshot_to_redshift( major_merger.merger_snapnum ),
                 z0_sid, 
                 j+1, 
                 major_mlpid, 
                 )

        data.append( _data )
    
dt = np.dtype([('lld_cb_om',float),
                ('lld_cb_self',float),
                ('lld_om_self',float),
                ('lld_cb_om_self',float),
                ('lld_N',int),
                ('star_mass',float),
                ('dm_mass',float),
                ('star_mass_ratio',float),
                ('dm_mass_ratio',float),
                ('z_merger',float),
                ('z0_sid',int),
                ('major_merger',int),
                ('major_mlpid',int),
                ])
lldata = np.array(data,dtype=dt)
os.makedirs('./data/', exist_ok=True)
np.save('./data/lldata.npy',lldata)

### Plot the number-weighted $\mathcal{L}$ differences for each DF

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

x = lldata['lld_cb_om']/lldata['lld_N']
ax.hist(x, range=[-1,1], bins=20, histtype='step', color='k', lw=2 )
_y = np.ones_like(x)

ax = pplot.plot_elements_out_of_bounds_as_arrows(x, _y, ax, 
    arrow_with_count=True, fx=0.1, fy=0.1)

ax.axvline(0., color='k', ls='--', lw=2)
ax.set_xlabel(r'$\frac{1}{N} \Delta \log \mathcal{L}_{\rm CB-OM}$')
ax.set_ylabel('N')
fig.tight_layout()

print(ax.get_xlim())

In [ ]:
fig = plt.figure(figsize=(6,8))
axs = fig.subplots(nrows=3, ncols=1)

x1 = lldata['lld_cb_om_self']/lldata['lld_N']
axs[0].hist(x1, range=[-1,1], bins=20, histtype='step', color='k', lw=2 )
_y = np.ones_like(x1)
axs[0] = pplot.plot_elements_out_of_bounds_as_arrows(x1, _y, axs[0], 
    arrow_with_count=True, fx=0.1, fy=0.1)
axs[0].axvline(0., color='k', ls='--', lw=2)
axs[0].set_xlabel(r'$\frac{1}{N} \Delta \log \mathcal{L}_{\rm CB-OM-SIM}$')
axs[0].set_ylabel('N')

x2 = lldata['lld_cb_self']/lldata['lld_N']
axs[1].hist(x2, range=[-1,1], bins=20, histtype='step', color='k', lw=2 )
_y = np.ones_like(x2)
axs[1] = pplot.plot_elements_out_of_bounds_as_arrows(x2, _y, axs[1], 
    arrow_with_count=True, fx=0.1, fy=0.1)
axs[1].axvline(0., color='k', ls='--', lw=2)
axs[1].set_xlabel(r'$\frac{1}{N} \Delta \log \mathcal{L}_{\rm CB-SIM}$')
axs[1].set_ylabel('N')

x3 = lldata['lld_om_self']/lldata['lld_N']
axs[2].hist(x3, range=[-1,1], bins=20, histtype='step', color='k', lw=2 )
_y = np.ones_like(x3)
axs[2] = pplot.plot_elements_out_of_bounds_as_arrows(x3, _y, axs[2], 
    arrow_with_count=True, fx=0.1, fy=0.1)
axs[2].axvline(0., color='k', ls='--', lw=2)
axs[2].set_xlabel(r'$\frac{1}{N} \Delta \log \mathcal{L}_{\rm OM-SIM}$')
axs[2].set_ylabel('N')

fig.tight_layout()
fig.show()

In [ ]:
facecolor='none'
edgecolor='k'
s = 10

fig = plt.figure(figsize=(12,4))
axs = fig.subplots(nrows=1, ncols=4)

xlabels = [r'$M_{\star}$', r'$z_{\rm merger}$', 
           r'$M_{\rm \star,p}/M_{\rm \star,s}$', 
           r'$M_{\rm DM,p}/M_{\rm DM,s}$']

ys = [lldata['star_mass'], lldata['z_merger'], 
      1/lldata['star_mass_ratio'], 1/lldata['dm_mass_ratio']]
x = lldata['lld_cb_om']/lldata['lld_N']

for k in range(4):
    axs[k].scatter(ys[k], x, facecolor=facecolor, edgecolor=edgecolor, s=s, 
        zorder=2)
                  
    axs[k].set_xlabel(xlabels[k])
    axs[k].set_ylim(-1.5,1)
    axs[k].axhline(0., color='Gray', ls='--', lw=1, zorder=1)
    arrow_kwargs = {'alpha':0.5}
    axs[k] = pplot.plot_elements_out_of_bounds_as_arrows(ys[k], x, axs[k], 
        arrow_with_count=False, fx=0.1, fy=0.1, arrow_kwargs=arrow_kwargs)
    if k in [0,2,3]:
        axs[k].set_xscale('log')
    if k in [1,2,3]:
        axs[k].tick_params(labelleft=False)
    
    indep_argsort = np.argsort(ys[k])
    window_size=15
    ma_x = np.convolve(ys[k][indep_argsort], 
        np.ones(window_size)/window_size, mode='valid')
    ma_y = np.convolve(x[indep_argsort], 
        np.ones(window_size)/window_size, mode='valid')
    axs[k].plot(ma_x, ma_y, color='Red', ls='solid', lw=1, zorder=3)

axs[0].set_ylabel(r'$\frac{1}{N} \Delta \log \mathcal{L}_{\rm CB-OM}$')

fig.tight_layout()
plt.show()